## The principles of functional programming

- *Functions as values*
- *Pure functions*
- *Immutability of data*
- *Composition*
- *Referential  transparency*
- *Type system*

## Functions as values

Functions are *objects* of the programming language.  They can be assigned and manipulated like other values:



In [ ]:

def x()-> int:
    return 3

y = x




This defines a zero-argument function `x` that returns an integer.
We then assign the value of `x` to `y` and check whether they are the same function:

In [ ]:
print(x() == y())

Success! We see that functions are values and can be used as any other value.

## Higher order functions

The fact that functions are values leads to the use *higher-order-functions*, which take other functions as parameter. This idiom is useful to abstract over the behavior of programs.




For example, `map` is used to apply a function to a list of values:

In [ ]:
from IPython.display import Markdown
import inspect
def my_fun(a: int)-> int:
    return a + 1

input_list = list(range(10))
result = list(map(my_fun, input_list))



In [ ]:
print(f"Input: {input_list}, output: {result}")

We can create *anonymous* functions using `lambda`. These are functions that do not have a name like in `def`, and are often used for *throwaway* functions that we need to use only once inside of another function. 
We can name them by assigning them to a variable:

In [ ]:
new_fun = lambda x: x + 1

In [ ]:
new_fun(5)

## Pure functions

We prefer functions that have no *side-effects*: 

1. they do not modify the state of the program 
2. given the same input, they **always** return the same output.



This following function is not *pure* because it modifies the value of `x`, defined outside of its *scope* every time it runs:


In [ ]:
from typing import Any
x = [1,2,3]
def do_something(val: int) -> list[int]:
    return x.append(val)

def display_val(x: Any):
    print(f"{x=}, {id(x)=}")


In [ ]:
display_val(x)
do_something(1)
display_val(x)
do_something(2)
display_val(x)

`id(x)` returns the memory address of a variable. If it does not change, it means the variable refers to the same object. This is the case here, therefore we proved that `do_something` is not *pure*.

## Immutable Data

Whenever possible, we avoid *mutating* data, as mutation is a type of *side-effect* and leads to unexpected behavior. 




- When a variable is defined, its value **remains constant for the duration of the program execution**.

- This makes it easier to reason about the flow of our program. 

For example, instead of inserting an element in an existing list as we did before, we create a new list by combining the old list with the new element:

In [ ]:
x = [1, 2, 3]
print(f"{x=}")
y = x + [4,]
print("Appending")
print(f"{x=}")
print(f"{y=}")

As you can see, the value of `x` does not change when we create `y`. This is in contrast to the previous example.

## List comprehensions
Immutability of data in python is frequently expressed using *comprehensions*.
These are for-like expressions that operate on iterables and produce *values* instead of modifying existing variables (no side effects).



To see how this works, let's first look at an example of *mutation*: we want to produce a list of the first 10 squares:

In [ ]:
#We define an empty list
a = []
#range(n) creates an iterable from 0,...,n
for i in range(10):
    #We append the current square to the existing list
    a.append(i**2)
    print(f"{a=}, {id(a)=}")

`id(a)` returns the memory *location* of `a`. As you can see, we modify the contents of `a` over the for loop: a always refers to the same location in the memory and therefore to the same object.

Let's rewrite this using a compherension and see what happens:

In [ ]:
#range(n) creates an iterable from 0,...,n
a = [i for i in range(10)]
print(f"{a=}, {id(a)=}")
b = [i**2 for i in a]
print(f"{b=}, {id(b)=}")
print(f"{a=}, {id(a)=}")

We see that the second expression produced a *new list* that we assigned  to `b`. The value of `a` remained unchanged.

## Composition
We build our programs by *composing* many simple functions togehter.

Why:


- It is easier to fix simple functions
- It is easier to optimise the program
- Single responbility: every function only does one job
- ...

How do we do this?

In [ ]:
# This is not ideal
def long_function(x: int) -> int:
    x1 = x + 2
    x2 = x1 - 4
    x3 = x2**2
    return x3

Instead of running all computation in a single long function, we can do better by *composing* many small functions.
 

In [ ]:
def f1(x: int) -> int:
    return x + 2

def f2(x: int) -> int:
    return x - 4

def f3(x: int) -> int:
    return x ** 2

# `better_function` is the "composition" of f3, f2, f1
better_function = lambda x: f3(f2(f1(x)))



In [ ]:
better_function(3) == long_function(3)

Instead of writing a long function, we broke it down into smaller functions and reconstructed it by composing them together. 


It must be clarified that with the long-function example, we try to explain the concept of single responsibility. Functions should solve a specific task and not try to do it all in one. Sometimes, however, one task requires many lines of code, and that is Ok.

## Referential transparency
This sounds scary, but it simply means that whenever we see a function call (or any expression), we can subsitute it with the value of the function.

This is the essence of functional programming: *there are no surprises*. We can reason about the flow of our program 

Consider this example:

In [ ]:

def transparent_fun(x: int) -> int:
    return x + 2

y1 = transparent_fun(1) + 3

x_out = transparent_fun(1)
y2 = x_out + 3



In [ ]:
print(f"{y1=}")
print(f"{y2=}")


We see that `transparent_function` is indeed referentially transparent. Now, look at another example:

In [ ]:
import random
def non_transparent_function(x: int) -> int:
    #This generates a random integer between 0 and 10
    return x + random.randint(0, 10)

# This "captures" the expression x() + 3 without computing its value by defining an "anonymous function"
# We get the value by calling y1()
y1 = lambda x: non_transparent_function(x) + 3





In [ ]:
print(f"{y1()=}")
print(f"{y1()=}")

This is *not*. Every time we call `non_transparent_function`, we get a different (random) value. 



In general, any expression containing a *combination of pure functions* and no side-effect is referentially transparent. We try to write our programs by composing referentially transparent functions as much as possible.

Why:
- We can reason about the program flow
- Compilers and other tools can reason for us and help finding errors
- It is easier to optimise our programs

## Type system

A typical trait of (modern) functional programming, is the heavy use of the *type system*.
This provides a set of rules to classify expression and values in the language into classes called *types*.
You encountered some basic types in the introduction, for example `int`, `str` or `float`.


We use type systems to enforce constraints and invariants in our program:

- *Making illegal states unrepresentable*
- Making sure we pass the right input to functions
- Checking if we wrote a valid program before it runs



Modern (3.6+) python allows using *type annotation* to mark the type of values, function parameters and class members.

For example:

```
def fun(a: int) -> str:
    return f"{a}"
```

means that `fun` takes integer values and returns strings. Python *does not enforce* this constraint. There are tools however, which can be used to check the conistency of types in a python program.

For example, instead of doing:

In [ ]:
#This function greets a person by name and age
def greet(name: str, age: int):
    print(f"Hello {name}, you are {age} years old")
    


In [ ]:
greet("Simone", 34)

Because python lacks static type checking, we can do absurd things:

In [ ]:
greet("Simone", "age")

We can do better by using *classes* to simulate the stricter type checking of other languages.

If you don't know what a class is, consider it as a sort of container type to which we can attach functionalities. We will see this in more detail in another lesson.

In [ ]:

#This defines a simple class with some helper methods and validations
#in this way, we are sure that when we pass a person to the `greet_better` function, this
#person will have all needed attributes
class Person:
    name: str
    age: int
    def __init__(self, name: str, age: int):
        if not isinstance(name, str) or not isinstance(age, int):
            raise ValueError("Invalid input")
        self.age = age
        self.name = name

def greet_better(person: Person):
    if isinstance(person, Person):
        print(f"Hello {person.name}, you are {person.age} years old")
    else:
        raise ValueError("Not a person")


In [ ]:
greet_better(Person("Simone", 34))

try:
    greet_better(Person("Simone", "d"))
except ValueError as e:
    print(f"Ooops, it does not work: {e}")
    
try:
    greet_better(("Simone", "d"))
except ValueError as e:
    print(f"Ooops, it does not work: {e}")